In [1]:
import time

In [2]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('wiki-changes-event-consumer')
         # Add kafka package
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
         .getOrCreate())
sc = spark.sparkContext

:: loading settings :: url = jar:file:/Users/shriyamishra/miniconda3/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/shriyamishra/.ivy2/cache
The jars for the packages stored in: /Users/shriyamishra/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a0b7fd94-b811-4306-a7ad-bcbff5314605;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.1 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 784ms :: artifacts dl 29ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.

In [3]:
# Create stream dataframe setting kafka server, topic and offset option
df = (spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", "localhost:9092") # kafka server
  .option("subscribe", "movielog22") # topic
  .option("startingOffsets", "latest") # start from beginning 
  .load())

In [4]:
readable = df.selectExpr("CAST(value AS STRING)")

In [5]:
raw_path = "/Users/shriyamishra/L3/spark-kafka-consumer"
checkpoint_path = "/Users/shriyamishra/L3/spark-kafka-consumer"

In [6]:
query = readable  \
    .writeStream  \
    .outputMode("append")  \
    .option("checkpointLocation", checkpoint_path)\
    .option("path", raw_path)\
    .format("parquet")  \
    .option("truncate", "False")  \
    .start()
time.sleep(10)    
query.stop()

22/03/16 19:12:04 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/03/16 19:12:15 ERROR FileFormatWriter: Aborting job e333d782-11bb-4153-bc19-a4b1a39291ef.
java.lang.InterruptedException
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.doAcquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1040)
	at java.base/java.util.concurrent.locks.AbstractQueuedSynchronizer.acquireSharedInterruptibly(AbstractQueuedSynchronizer.java:1345)
	at scala.concurrent.impl.Promise$DefaultPromise.tryAwait(Promise.scala:242)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:258)
	at scala.concurrent.impl.Promise$DefaultPromise.ready(Promise.scala:187)
	at org.apache.spark.util.ThreadUtils$.awaitReady(ThreadUtils.scala:334)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:929)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2214)
	at org.apache.spark.sql.

In [7]:
df_read=spark.read.parquet(raw_path)

In [8]:
df_read.take(10)

[Row(value='2022-03-16T19:03:09,607298,GET /data/m/schindlers+list+1993/1.mpg'),
 Row(value='2022-03-16T19:03:09,624547,GET /data/m/dances+with+wolves+1990/103.mpg'),
 Row(value='2022-03-16T19:03:09,681735,GET /data/m/pather+panchali+1955/7.mpg'),
 Row(value='2022-03-16T19:03:09,688475,GET /data/m/eraser+1996/16.mpg'),
 Row(value='2022-03-16T19:03:09,455089,GET /data/m/when+we+were+kings+1996/21.mpg'),
 Row(value='2022-03-16T19:03:09,651946,GET /data/m/dances+with+wolves+1990/24.mpg'),
 Row(value='2022-03-16T19:03:09,589532,GET /data/m/citizen+kane+1941/35.mpg'),
 Row(value='2022-03-16T19:03:09,463014,GET /data/m/kind+hearts+and+coronets+1949/44.mpg'),
 Row(value='2022-03-16T19:03:09,388333,GET /data/m/without+a+paddle+2004/28.mpg'),
 Row(value='2022-03-16T19:03:09,22036,GET /data/m/nothing+lasts+forever+1984/11.mpg')]